In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install tensorflow
!pip install numpy pandas scikit-learn
!pip install pyrouge

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191604 sha256=4689c071bec610c674f9923b709785772014c16a051f45daed7363bb2c6bd2ab
  Stored in directory: /root/.cache/pip/wheels/9a/67/12/c5dd8ef8b4152bb8789eafd2a74a734e2dc7bb9eae02b768e7
Successfully built pyrouge


In [ ]:
!git clone https://github.com/ThanhChinhBK/vietnews.git

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Total 143827 (delta 0), reused 0 (delta 0), pack-reused 143827 (from 1)
Receiving objects: 100% (143827/143827), 194.68 MiB | 17.14 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Updating files: 100% (150704/150704), done.


In [ ]:
import os
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
import glob

In [ ]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles


In [ ]:
train_paths = listPaths('/content/vietnews/data/train_tokenized/*')[:200]
val_paths = listPaths('/content/vietnews/data/val_tokenized/*')[:100]
test_paths = listPaths('/content/vietnews/data/test_tokenized/*')[:100]

In [ ]:
def read_content(pathfile):
    try:
        with open(pathfile, encoding='utf-8') as f:
            rows = f.readlines()
            original = ' '.join(''.join(rows[4:]).split('\n'))
            summary = ' '.join(rows[2].split('\n'))


        return {
            'file': pathfile,
            'original': original,
            'summary': summary
        }
    except Exception as e:
        print(f"Lỗi khi xử lý {pathfile}: {e}")
        return None

In [ ]:
read_content(train_paths[0])

{'file': '/content/vietnews/data/train_tokenized/003669.txt.seg',
 'original': 'Ngày 28/8 , TAND tỉnh Nghệ_An mở phiên_toà hình_sự sơ_thẩm xét_xử bị_cáo Lê_Quang_Hoà ( 39 tuổi ) , trú xã Bồng_Khê , huyện Con_Cuông , tỉnh Nghệ_An về tội Lừa_đảo chiếm_đoạt tài_sản . Để lừa_đảo , bị_cáo Lê_Quang_Hoà đã “ nổ ” mình là cán_bộ sở Nội_vụ tỉnh Nghệ_An hoặc Cán_bộ Công_an TP._Vinh , có mối quan_hệ thân_thiết với những người có chức_vụ và quyền_hạn trong các Sở , ban , ngành của UBND tỉnh Nghệ_An . Hoà cho biết mình có_thể xin việc cho nhiều người vào các cơ_quan Nhà_nước trong tỉnh , xin thuyên_chuyển công_tác từ các tỉnh , thành trên toàn_quốc về Nghệ_An . Qua một người giới_thiệu , Hoà biết bà Nguyễn_Thị_Minh_H. , trú huyện Hương_Khê ( Hà_Tĩnh ) có con_gái tốt_nghiệp đại_học nhưng chưa có việc_làm . Ngày 3/10/2017 , người phụ_nữ này đã gặp Hoà đề nhờ xin việc cho con_gái . Gặp bà H. , Hoà giới_thiệu mình là cán_bộ của sở Nội_vụ tỉnh Nghệ_An và cam_kết xin được_việc cho con_gái bà H. vào làm k

In [ ]:
train_data = [read_content(fp) for fp in train_paths if read_content(fp) is not None]
val_data = [read_content(fp) for fp in val_paths if read_content(fp) is not None]
test_data = [read_content(fp) for fp in test_paths if read_content(fp) is not None]

In [ ]:
class TextSummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=512, max_output_length=150):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source_text = self.data[idx]['original']
        target_text = self.data[idx]['summary']

        inputs = self.tokenizer(
            source_text, max_length=self.max_input_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        targets = self.tokenizer(
            target_text, max_length=self.max_output_length, padding="max_length", truncation=True, return_tensors="pt"
        )

        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        labels = targets["input_ids"].squeeze()

        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [ ]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
train_dataset = TextSummarizationDataset(train_data, tokenizer)
val_dataset = TextSummarizationDataset(val_data, tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.154877
2,No log,1.687602
3,No log,1.658538


TrainOutput(global_step=150, training_loss=2.7535359700520834, metrics={'train_runtime': 2172.0053, 'train_samples_per_second': 0.276, 'train_steps_per_second': 0.069, 'total_flos': 81205080883200.0, 'train_loss': 2.7535359700520834, 'epoch': 3.0})

In [ ]:
test_dataset = TextSummarizationDataset(test_data, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [ ]:
for batch in test_dataloader:
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]

    # Tạo dự đoán
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150)
    predicted_summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # In kết quả tóm tắt
    for i, summary in enumerate(predicted_summaries):
        print(f"Original: {test_data[i]['original']}")
        print(f"Predicted Summary: {summary}")
        print(f"Actual Summary: {test_data[i]['summary']}\n")

Original: Toàn_bộ tiền mua nhớt và các vật_dụng đều do các bạn trẻ cùng nhau đóng_góp . Những bạn làm nghề sửa xe_gắn_máy thì tình_nguyện góp công . Đến 11h trưa 7-8 , đã có hơn 100 xe_máy ngập nước của bà_con đưa đến được các bạn trẻ thay nhớt , làm vệ_sinh và sửa_chữa . Lượng người đưa xe đến quá đông trong khi khu_vực mượn tạm để sửa_chữa xe_hơi chật nên các bạn trẻ phải làm_việc hết công_suất . Anh Đinh_Quang_Thiều - trưởng nhóm - cho_biết hôm_qua anh_em đã liên_hệ UBND thị_trấn Dương_Đông mượn đường_băng sân_bay cũ để làm cho rộng . Tuy_nhiên vừa dựng rạp lên thì bị gió hất tung nên nhóm phải mượn tạm mái_hiên của một cửa_hàng vật_liệu xây_dựng ở đường Hùng_Vương để làm . Tối 6-8 , khi anh Thiều và nhóm bạn của mình đưa thông_tin địa_điểm thay nhớt , sửa xe miễn_phí lên mạng xã_hội , nhiều bà_con ở các khu_vực vô_cùng phấn_khởi . Tuy_nhiên vẫn có người băn_khoăn là làm_sao biết người nào khó_khăn thật_sự để thay nhớt miễn_phí , anh Thiều trả_lời : " Dễ lắm ! Đó là lòng tự_trọng ! 